In [37]:
import numpy as np
import random

class Node:
    def __init__(self, id):
        self.id = id
        self.position = None
        self.neighbors = None
    
    def show_param(self):
        print(self.id, self.position, self.neighbors)

    def get_position(self):
        x = random.randint(-100, 100)
        y = random.randint(-100, 100)
        z = random.randint(0, 50)
        self.position = np.asarray([x, y, z])
    
    def get_connections(self, num_cit, possibility):
        con = []
        for i in range(num_cit):
            if i == self.id:
                pass
            else:
                a = random.randint(1, 100)
                if a <= possibility:
                    con.append(i)
        self.neighbors = con
    
num_cit = 5
cities_lst = []
for i in range(num_cit):
    node = Node(i)
    node.get_position()
    node.get_connections(5, 90)
    node.show_param()
    cities_lst.append(node)

def calculate_cost(routes, cities, assimetrial=False):
    costs = []
    window = 2
    if assimetrial == False:
        for route in routes:
            cost = 0
            for i in range(len(route)):
                if route[i] == route[-1]:
                    cost = cost + np.linalg.norm(cities[route[-1]].position - cities[route[0]].position)
                    costs.append(cost)   
                else:
                    for j in range(len(route)):
                        a = route[j:j+window]
                        if len(a) == 2:
                            cost = cost + np.linalg.norm(cities[a[0]].position - cities[a[1]].position) 
        return costs
    elif assimetrial != False:
        for route in routes:
            cost = 0
            for i in range(len(route)):
                if route[i] == route[-1]:
                    direction = cities[route[-1]].position[2] - cities[route[0]].position[2]
                    if direction > 0 or direction < cities[route[-1]].position[2]:
                        cost = cost + (np.linalg.norm(cities[route[-1]].position - cities[route[0]].position)-np.linalg.norm(cities[route[-1]].position - cities[route[0]].position)*0.1)
                        costs.append(cost)   
                    else: 
                        cost = cost + (np.linalg.norm(cities[route[-1]].position - cities[route[0]].position)+np.linalg.norm(cities[route[-1]].position - cities[route[0]].position)*0.1)
                        costs.append(cost)  
                else:
                    for j in range(len(route)):
                        a = route[j:j+window]
                        if len(a) == 2:
                            direction = cities[a[0]].position[2] - cities[a[1]].position[2]
                            if direction > 0 or direction < cities[0].position[2]:
                                cost = cost + (np.linalg.norm(cities[a[0]].position - cities[a[1]].position)-np.linalg.norm(cities[a[0]].position - cities[a[1]].position)*0.1) 
                            else:
                                cost = cost + (np.linalg.norm(cities[a[0]].position - cities[a[1]].position)+np.linalg.norm(cities[a[0]].position - cities[a[1]].position)*0.1)
        return costs

0 [ 68 -10  16] [1, 2, 3, 4]
1 [ 72 -77   6] [0, 2, 3, 4]
2 [  0 -51   0] [0, 1, 3, 4]
3 [ 4 66 50] [0, 1, 2, 4]
4 [ 25 -83  22] [0, 1, 2, 3]


In [49]:
def dfs(city_map, start_point, path=None, path_lst=[]):
    if path is None:
        path = [start_point]

    if len(path) == len(city_map):
        path_lst.append(path)

    possible = city_map[start_point].neighbors
    for i in possible:
        if i not in path:
            new_path = path.copy()
            new_path.append(i)
            dfs(city_map, i, new_path, path_lst)

    return path_lst
a = dfs(cities_lst, 0)
print(len(a))




24


In [39]:
print(calculate_cost(a, cities_lst, assimetrial=False))
#cities_lst[2].position
lst = [1,2,3,4,5]
window = 2
for i in range(len(lst)):
    print(lst[i:i+window])

[1784.9351575393503, 1480.5563808186137, 1707.6855707312018, 1806.777039339286, 1270.4279776137457, 1673.8982229425662, 1985.6757168796057, 1548.418123762149, 1775.547313674737, 1713.3177720146837, 1471.1685369540007, 1846.1965884114034, 1654.2483257785866, 1543.2113911818024, 1521.3695093818667, 1381.8903809136648, 1620.460977989951, 1592.0187841185332, 1461.1324635650979, 1787.3531220857699, 1594.0112799618175, 1758.910928214352, 1997.4815252906383, 1836.1605150225007]
[1, 2]
[2, 3]
[3, 4]
[4, 5]
[5]


In [48]:
from collections import deque

def bfs_paths(graph, start):
    # Initialize the queue with the starting node and an empty path
    queue = deque([(start, [start])])
    all_paths = []

    while queue:
        current_node, path = queue.popleft()

        # Check if the current path reaches an endpoint (all nodes visited)
        if len(path) == len(graph):
            all_paths.append(path)
            continue

        for neighbor in graph[current_node].neighbors:
            if neighbor not in path:
                # Create a new path with the neighbor node added
                new_path = path + [neighbor]
                queue.append((neighbor, new_path))

    return all_paths

# Test the function with the provided graph
graph = {0: [1, 2, 3], 1: [0, 2, 3], 2: [0, 1, 3], 3: [0, 1, 2]}
start_node = 0
result = bfs_paths(cities_lst, start_node)
print(len(result))
print(calculate_cost(result, cities_lst, assimetrial=False))

24
[1784.9351575393503, 1480.5563808186137, 1707.6855707312018, 1806.777039339286, 1270.4279776137457, 1673.8982229425662, 1985.6757168796057, 1548.418123762149, 1775.547313674737, 1713.3177720146837, 1471.1685369540007, 1846.1965884114034, 1654.2483257785866, 1543.2113911818024, 1521.3695093818667, 1381.8903809136648, 1620.460977989951, 1592.0187841185332, 1461.1324635650979, 1787.3531220857699, 1594.0112799618175, 1758.910928214352, 1997.4815252906383, 1836.1605150225007]
